In [4]:
# describe how data was pre-processed and show sample output 

# Combine all datasets and tokenize text using the BERT tokenizer
# Create a dataset by combining the previous, current, and next sentences together. 
# If the current sentence is the first or last sentence of the file, then an empty string is 
# used as the previous or next sentence. 
# Data split into training and test sets.

In [ ]:
# description of the 4 networks (including the baseline one)

# Baseline: [TO DO]
# Network1: Network with 3 BERT embedding layers, taking target, preceding, and following sentence.
    # Embeds three sentences.
    # Passes pooled outputs (CLS tokens) of each sentence as a sequence to a Bi-LSTM. 
    # Takes middle hidden state of Bi-LSTM and passes through a two layer MLP.
# Network2: Network with 3 BERT embedding layers, taking target, preceding, and following sentence.
    # Embeds three sentences using BERT transformers.
    # Passes pooled outputs (CLS tokens) into two attention heads, computing attention between (S_T, S_P) and (S_T, S_F). 
    # Concatenates outputs of attention heads and passes through a two layer MLP.
# Network3: Network with 3 BERT embedding layers, taking target, preceding, and following sentence.
    # Embeds three sentences using BERT transformers.
    # Passes pooled outputs (CLS tokens) into two attention heads, computing attention between (S_T, S_P) and (S_T, S_F). 
    # Concatenates outputs of attention heads and passes through a two layer MLP.
    

In [5]:
# Trained the networks using train.py script.
# Output the print statements from training.

In [6]:
# Evaluated the performance of each network using precision, recall, f1, and accuracy; weighted and unweighted except accuracy.
# Potentially could just run that file here.
# Output the print statements/dataframes from the eval_metrics.py file

In [ ]:
# Maybe a markdown for any other things that should be noted when trying to reproduce 